# **Final LightGBM Model:** Hospital Charge Prediction

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import joblib

In [ ]:
df = pd.read_csv('../data/model_input_final_clean.csv')

In [38]:
# Define feature columns
feature_cols = [
    'gender_encoded', 'age_group_encoded', 'severity_encoded',
    'admission_encoded', 'payment_type_encoded', 'diagnosis_encoded',
    'procedure_encoded', 'county_encoded', 'los',
    'los_x_severity', 'los_x_procedure', 'severity_x_procedure', 'los_x_county',
    'diagnosis_x_severity', 'procedure_x_severity', 
    'diagnosis_x_procedure', 'county_x_los'
]

In [39]:
X = df[feature_cols]
y = np.log1p(df['total_charges'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
base_model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05)
base_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2544
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 17
[LightGBM] [Info] Start training from score 10.142437


LGBMRegressor(learning_rate=0.05, n_estimators=1000)

In [41]:
# Predict and evaluate on the test set
y_pred_log = base_model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # undo the log1p transformation
y_true = np.expm1(y_test)

# Calculate MAE
mae_global = mean_absolute_error(y_true, y_pred)
print(f"Global Model MAE: ${mae_global:,.2f}")

Global Model MAE: $13,057.92


In [43]:
# Add diagnosis group manually
def categorize_diagnosis(desc):
    desc = desc.lower()
    if any(x in desc for x in ['leukemia', 'lymphoma', 'myeloma', 'neoplasm', 'cancer']):
        return 'Oncology'
    elif any(x in desc for x in ['short gestation', 'liveborn', 'fetal', 'newborn', 'neonatal']):
        return 'Neonatal'
    elif any(x in desc for x in ['schizophrenia', 'mood', 'psychotic', 'substance', 'alcohol']):
        return 'Behavioral'
    elif any(x in desc for x in ['septicemia', 'pneumonia', 'infection']):
        return 'Infectious'
    elif any(x in desc for x in ['osteoarthritis', 'spondylosis', 'back problems']):
        return 'Orthopedic'
    elif any(x in desc for x in ['diabetes', 'heart failure', 'dysrhythmia', 'myocardial', 'coronary']):
        return 'Cardio-Metabolic'
    elif any(x in desc for x in ['pregnancy', 'delivery', 'c-section', 'puerperium']):
        return 'OB/GYN'
    else:
        return 'General'

# Load mapping (you need this .csv)
diag_map = pd.read_csv('../data/diagnosis_mapping.csv')

# Merge readable diagnosis names
df = df.merge(diag_map, on='diagnosis_encoded', how='left')

# Create group
df['diagnosis_group'] = df['ccs_diagnosis_description'].apply(categorize_diagnosis)

In [44]:
from collections import defaultdict
import lightgbm as lgb

group_models = {}
group_maes = {}

# Pull distinct groups
groups = df['diagnosis_group'].dropna().unique()

for group in groups:
    group_df = df[df['diagnosis_group'] == group].copy()

    X_group = group_df[feature_cols]
    y_group = np.log1p(group_df['total_charges'])
    X_train, X_test, y_train, y_test = train_test_split(X_group, y_group, random_state=42)
   
    model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05)
    model.fit(X_train, y_train)

    preds = np.expm1(model.predict(X_test))
    y_true = np.expm1(y_test)
    mae = mean_absolute_error(y_true, preds)

    group_models[group] = model
    group_maes[group] = mae

# Show all group MAEs
for group, mae in group_maes.items():
    print(f"{group} MAE: ${mae:,.2f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2544
[LightGBM] [Info] Number of data points in the train set: 833469, number of used features: 17
[LightGBM] [Info] Start training from score 10.284842
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001960 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1906
[LightGBM] [Info] Number of data points in the train set: 161873, number of used features: 17
[LightGBM] [Info] Start training from score 10.396232
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

In [45]:
# Start with global model predictions
df['ensemble_prediction'] = np.expm1(base_model.predict(df[feature_cols]))
df['hybrid_prediction'] = df['ensemble_prediction'].copy()

# Baseline MAEs for comparison
original_group_maes = {
    'Oncology': 19382,
    'Orthopedic': 15300,
    'Cardio-Metabolic': 15100,
    'General': 13700,
    'Infectious': 13300,
    'Behavioral': 7600,
    'Neonatal': 5200,
    'OB/GYN': 4400,
}

# Replace with group predictions only where they improve
for group in df['diagnosis_group'].dropna().unique():
    if group in original_group_maes and group_maes[group] < original_group_maes[group]:
        print(f"✅ Using grouped model for {group} (MAE improved)")
        idx = df['diagnosis_group'] == group
        df.loc[idx, 'hybrid_prediction'] = np.expm1(group_models[group].predict(df.loc[idx, feature_cols]))

In [46]:
final_mae = mean_absolute_error(df['total_charges'], df['hybrid_prediction'])
print(f"\n🏁 Final Hybrid Model MAE: ${final_mae:,.2f}")


🏁 Final Hybrid Model MAE: $13,049.32


In [48]:
# Define X and y
X = df[feature_cols]
y = np.log1p(df['total_charges'])  # log-transform target for training

# Train-validation split for tuning
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
import optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "random_state": 42,
        "n_jobs": -1,
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)

    preds = np.expm1(model.predict(X_valid))
    true = np.expm1(y_valid)
    mae = mean_absolute_error(true, preds)
    return mae

In [50]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best MAE:", study.best_value)
print("Best Parameters:", study.best_params)

[I 2025-08-05 23:23:49,019] A new study created in memory with name: no-name-c3f7768a-5c8b-4e65-871b-c4ded9f1318b


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010759 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2531
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 17
[LightGBM] [Info] Start training from score 10.142437
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2025-08-05 23:24:11,712] Trial 0 finished with value: 15796.311955832529 and parameters: {'n_estimators': 885, 'learning_rate': 0.012580626130698951, 'max_depth': 4, 'num_leaves': 83, 'min_child_samples': 83, 'subsample': 0.6974600587629703, 'colsample_bytree': 0.5147787808493577, 'reg_alpha': 1.329715990334885, 'reg_lambda': 1.9529126832907784}. Best is trial 0 with value: 15796.311955832529.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2531
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 17
[LightGBM] [Info] Start training from score 10.142437
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2025-08-05 23:25:38,427] Trial 1 finished with value: 12289.549010091927 and parameters: {'n_estimators': 1818, 'learning_rate': 0.2824995389326457, 'max_depth': 10, 'num_leaves': 247, 'min_child_samples': 72, 'subsample': 0.8435801249345006, 'colsample_bytree': 0.932186145445216, 'reg_alpha': 2.2682553982381344, 'reg_lambda': 1.3671269255080776}. Best is trial 1 with value: 12289.549010091927.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2531
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 17
[LightGBM] [Info] Start training from score 10.142437
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-08-06 00:27:47,266] Trial 48 finished with value: 12544.459317711899 and parameters: {'n_estimators': 1280, 'learning_rate': 0.015712119686297488, 'max_depth': 12, 'num_leaves': 251, 'min_child_samples': 68, 'subsample': 0.8235909698345547, 'colsample_bytree': 0.6585084416075615, 'reg_alpha': 3.0420394720459822, 'reg_lambda': 0.6148182164006577}. Best is trial 36 with value: 12020.217930996154.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2531
[LightGBM] [Info] Number of data points in the train set: 1873243, number of used features: 17
[LightGBM] [Info] Start training from score 10.142437
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[I 2025-08-06 00:29:22,183] Trial 49 finished with value: 12047.369674419511 and parameters: {'n_estimators': 1746, 'learning_rate': 0.1011762380093266, 'max_depth': 10, 'num_leaves': 273, 'min_child_samples': 58, 'subsample': 0.7786449329510048, 'colsample_bytree': 0.7546018611043213, 'reg_alpha': 2.473721369895843, 'reg_lambda': 1.7904036395754952}. Best is trial 36 with value: 12020.217930996154.


Best MAE: 12020.217930996154
Best Parameters: {'n_estimators': 1874, 'learning_rate': 0.0875403852825351, 'max_depth': 12, 'num_leaves': 265, 'min_child_samples': 61, 'subsample': 0.6745113925252707, 'colsample_bytree': 0.6858714521709702, 'reg_alpha': 3.4299949019901, 'reg_lambda': 1.4416258237066328}


In [51]:
study.best_params

{'n_estimators': 1874,
 'learning_rate': 0.0875403852825351,
 'max_depth': 12,
 'num_leaves': 265,
 'min_child_samples': 61,
 'subsample': 0.6745113925252707,
 'colsample_bytree': 0.6858714521709702,
 'reg_alpha': 3.4299949019901,
 'reg_lambda': 1.4416258237066328}

In [52]:
final_model = lgb.LGBMRegressor(
    n_estimators=1874,
    learning_rate=0.08754,
    max_depth=12,
    num_leaves=265,
    min_child_samples=61,
    subsample=0.6745,
    colsample_bytree=0.68587,
    reg_alpha=3.43,
    reg_lambda=1.44
)

X = df[feature_cols]
y = np.log1p(df['total_charges'])

final_model.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2540
[LightGBM] [Info] Number of data points in the train set: 2341554, number of used features: 17
[LightGBM] [Info] Start training from score 10.142602
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

LGBMRegressor(colsample_bytree=0.68587, learning_rate=0.08754, max_depth=12,
              min_child_samples=61, n_estimators=1874, num_leaves=265,
              reg_alpha=3.43, reg_lambda=1.44, subsample=0.6745)

In [ ]:
df['ensemble_prediction'] = np.expm1(final_model.predict(df[feature_cols]))
df['hybrid_prediction'] = df['ensemble_prediction'].copy()

# Use group models only if they beat the global one (already done earlier)
for group in groups:
    if group in original_group_maes and group_maes[group] < original_group_maes[group]:
        idx = df['diagnosis_group'] == group
        df.loc[idx, 'hybrid_prediction'] = np.expm1(group_models[group].predict(df.loc[idx, feature_cols]))